## Testing with unitest
We use testing to check the interactions between other already tested code.

https://docs.python.org/3/library/unittest.mock.html

`Mock` and `MagicMock` objects create all attributes and methods as you access them and store details of how they have been used. You can configure them, to specify return values or limit what attributes are available, and then make assertions about how they have been used:

In [3]:
from unittest.mock import Mock, MagicMock, patch
import pytest
import os
DOMAIN = 'fake'
EVENT_TYPE = 'deleted'
FILE = 'file.txt'
FOLDER = 'test'
SRC_PATH = 'test/file.txt'

In [2]:
# Create a fake class for example purposes

class ProductionClass():
    """A fake class."""
    def __init__(self):
        pass
    
    def method(self):
        pass

Mock the return of a class method

In [3]:
thing = ProductionClass()
thing.method = MagicMock(return_value=3)

In [4]:
thing.method(3, 4, 5, key='value')

3

Assert that the method has been called

In [5]:
thing.method.assert_called_with(3, 4, 5, key='value') # An error is raised if assert_called_with is false

#### Function generators

Commonn practice is to generate mock objects from functions with their attributes correctly configured.

In [6]:
def get_fake_event(src_path=SRC_PATH, event_type=EVENT_TYPE):
    """Generate a Fake watchdog event object with the specified arguments."""
    return MagicMock(
        src_path=src_path, event_type=event_type, is_directory=False)

In [7]:
fake_event = get_fake_event()

In [8]:
fake_event.src_path

'test/file.txt'

In [9]:
folder, file_name = os.path.split(fake_event.src_path)
print(folder)
print(file_name)

test
file.txt


In [10]:
fake_event.event_type

'deleted'

In [11]:
fake_event.is_directory

False

## Fake HASS

Just use Mock to test a method.

In [12]:
fake_hass = Mock()

In [13]:
fake_hass.bus.fire('test')

<Mock name='mock.bus.fire()' id='4370268848'>

In [14]:
fake_hass.bus.fire('test')

<Mock name='mock.bus.fire()' id='4370268848'>

In [15]:
payload = {"event_type": fake_event.event_type,
           'path': fake_event.src_path,
           'file': 'file.txt',
           'folder': 'test'}

fake_hass.bus.fire(DOMAIN, payload)

<Mock name='mock.bus.fire()' id='4370268848'>

In [16]:
fake_hass.bus.fire.assert_called_with(DOMAIN, payload)

## Modules

Mock a whole module

In [17]:
def test_import_patching():
    module_mock = MagicMock()
    with patch.dict('sys.modules', **{ 
            'unimportable_module': module_mock,
            'unimportable_module.submodule': module_mock,
        }):
        import unimportable_module.submodule
        assert unimportable_module.submodule == module_mock.submodule

## Mock Patch

https://blog.fugue.co/2016-02-11-python-mocking-101.html

The patch() decorator makes it easy to mock classes or objects in a module under test. The object you specify will be replaced with a mock (or other object) during the test and restored when the test ends:

As well as a decorator patch() can be used as a context manager in a with statement:

In [18]:
with patch.object(ProductionClass, 'method', return_value="my return") as mock_method:
    thing = ProductionClass()
    print(thing.method(1, 2, 3))

my return


In [19]:
test_import_patching()

## Hass

In [20]:
def create_event_handler(hass):
    """"Return the Watchdog EventHandler object."""

    class EventHandler():
        """Class for handling Watcher events."""

        def __init__(self, hass):
            """Initialise the EventHandler."""
            self.hass = hass

        def process(self, event):
            """On Watcher event, fire HA event."""
            if not event.is_directory:
                print('fire')
                folder, file_name = os.path.split(event.src_path)
                self.hass.bus.fire(
                    DOMAIN, {
                        "event_type": event.event_type,
                        'path': event.src_path,
                        'file': file_name,
                        'folder': folder,
                        })

    return EventHandler(hass)

In [21]:
fake_event = get_fake_event()
fake_hass = Mock()
event_handler = create_event_handler(fake_hass)

In [22]:
event_handler.process(fake_event)

fire


In [23]:
fake_hass.bus.fire.assert_called()

In [24]:
expected_payload = {"event_type": fake_event.event_type,
                    'path': fake_event.src_path,
                    'file': FILE,
                    'folder': FOLDER}

fake_hass.bus.fire.assert_called_with(DOMAIN, expected_payload)

## Pytest fixtures

https://docs.pytest.org/en/latest/fixture.html

Test functions can receive fixture objects by naming them as an input argument. For each argument name, a fixture function with that name provides the fixture object. Fixture functions are registered by marking them with @pytest.fixture.

In [5]:
@pytest.fixture
def smtp_connection():
    import smtplib
    return smtplib.SMTP("smtp.gmail.com", 587, timeout=5)

def test_ehlo(smtp_connection):
    response, msg = smtp_connection.ehlo()
    assert response == 250
    assert 0 # for demo purposes

In [6]:
test_ehlo()

TypeError: test_ehlo() missing 1 required positional argument: 'smtp_connection'

In [1]:
def validate_api_key(api_key):
    """Check that an API key is valid, if yes return the app."""
    try:
        from clarifai.rest import ClarifaiApp, ApiError
        app = ClarifaiApp(api_key=api_key)
        return app
    except ApiError as exc:
        error = json.loads(exc.response.content)
        _LOGGER.error(
            "%s error: %s", CLASSIFIER, error['status']['details'])
        return None

In [4]:
@pytest.fixture
def mock_clarifai():
    """Return a mock camera image."""
    with patch('clarifai.rest.ClarifaiApp') as _mock_clarifai:
        yield _mock_clarifai